# Data Preprocessing

##### Downloading dataset

In [17]:
import opendatasets as od
import pandas as pd

od.download('https://www.kaggle.com/datasets/shariful07/student-mental-health/data')

dataset = pd.read_csv("student-mental-health\\Student Mental health.csv")
dataset.head()

Skipping, found downloaded files in ".\student-mental-health" (use force=True to force download)


,Timestamp,Choose your gender,Age,What is your course?,Your current year of Study,What is your CGPA?,Marital status,Do you have Depression?,Do you have Anxiety?,Do you have Panic attack?,Did you seek any specialist for a treatment?
0,08/07/2020 12:02,Female,18.0,Engineering,year 1,3.00 - 3.49,No,Yes,No,Yes,No
1,08/07/2020 12:04,Male,21.0,Islamic education,year 2,3.00 - 3.49,No,No,Yes,No,No
2,08/07/2020 12:05,Male,19.0,BIT,Year 1,3.00 - 3.49,No,Yes,Yes,Yes,No
3,08/07/2020 12:06,Female,22.0,Laws,year 3,3.00 - 3.49,Yes,Yes,No,No,No
4,08/07/2020 12:13,Male,23.0,Mathemathics,year 4,3.00 - 3.49,No,No,No,No,No


In [18]:
#Dropping timestamp column
dataset = dataset.drop("Timestamp", axis=1) 

In [19]:
#Processing gender column
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset['Choose your gender'].fillna('Female', inplace=True) #Filling empty cells with value "Female"
dataset['Choose your gender']= label_encoder.fit_transform(dataset['Choose your gender'])

In [20]:
#Processing age column
mean_age = dataset['Age'].mean()
dataset['Age'].fillna(mean_age, inplace=True)
mean_age = round(mean_age) # Round the mean value to the nearest integer (optional)
dataset['Age'] = dataset['Age'].astype(int)# Convert the 'Age' column to integers

In [ ]:
#Processing course column


In [21]:
dataset.head()

,Choose your gender,Age,What is your course?,Your current year of Study,What is your CGPA?,Marital status,Do you have Depression?,Do you have Anxiety?,Do you have Panic attack?,Did you seek any specialist for a treatment?
0,0,18,Engineering,year 1,3.00 - 3.49,No,Yes,No,Yes,No
1,1,21,Islamic education,year 2,3.00 - 3.49,No,No,Yes,No,No
2,1,19,BIT,Year 1,3.00 - 3.49,No,Yes,Yes,Yes,No
3,0,22,Laws,year 3,3.00 - 3.49,Yes,Yes,No,No,No
4,1,23,Mathemathics,year 4,3.00 - 3.49,No,No,No,No,No
